# Compression Experiments WET <> Parquet

Analyze the impact on storage size by
- compression algorithm
- sorting by column

See also notes about [Parquet](../notes/parquet.md).

In [1]:
import os

from urllib.parse import urlparse

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import requests
import tldextract

# fastwarc installed as: pip3 install -U --no-binary fastwarc fastwarc
from fastwarc.warc import ArchiveIterator
from fastwarc.stream_io import FastWARCError


wet_file = 'CC-MAIN-20210804142918-20210804172918-00307.warc.wet.gz'
wet_url = 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2021-31/segments/1627046154878.27/wet/' + wet_file
if not os.path.exists(wet_file):
    r = requests.get(wet_url)
    open(wet_file, 'wb').write(r.content)

In [2]:
def print_file_size(file, compare_to=None):
    size = os.stat(file).st_size
    if compare_to:
        size2 = os.stat(compare_to).st_size
        print('{:4.3f} MiB  {:5.1f}%  {}'.format(size/2**20, 100.0*size/size2, file))
    else:
        print('{:4.3f} MiB  {}'.format(size/2**20, file))

print_file_size(wet_file)

138.047 MiB  CC-MAIN-20210804142918-20210804172918-00307.warc.wet.gz


In [3]:
wet_iter = ArchiveIterator(open(wet_file, 'rb'))

data = []

for rec in wet_iter:
    if rec.headers.get('WARC-Type') != 'conversion':
        continue
    date = rec.headers.get('WARC-Date')
    url = rec.headers.get('WARC-Target-URI')
    host = urlparse(url).netloc.split(':')[0]
    host_reversed_parts = list(filter(lambda x: x, reversed(host.split('.'))))
    host_reversed = '.'.join(host_reversed_parts)
    tld = host_reversed_parts[0]
    content_languages = rec.headers.get('WARC-Identified-Content-Language')
    content = rec.reader.read().decode('utf8')
    data.append([pd.Timestamp(date), url, tld, host, host_reversed, content_languages, content])

columns = ['date', 'url', 'tld', 'host', 'host_reversed', 'content_languages', 'content']
df = pd.DataFrame(data, columns=columns)

In [4]:
df.head()

,date,url,tld,host,host_reversed,content_languages,content
0,2021-08-04 15:52:31+00:00,http://000006a53723.hgcgsc.com/Front/Productbu...,com,000006a53723.hgcgsc.com,com.hgcgsc.000006a53723,zho,化工采购商城\n您好，欢迎光临南京杜瑞贸易有限公司！\n请登录\n免费注册\n购物车(0)\...
1,2021-08-04 15:36:38+00:00,http://000958.cn/,cn,000958.cn,cn.000958,zho,东方能源 - 股票000958.CN | 主营清洁能源发电及热电联产的公司\n东方能源(00...
2,2021-08-04 16:51:43+00:00,http://01umz.com/index.php/vod/detail/id/4276....,com,01umz.com,com.01umz,"zho,jpn,eng",APP下载\n独家经典\n国产精品\n日韩无码\n欧美精品\n日韩精品\n人妻系列\n伦理影...
3,2021-08-04 15:38:07+00:00,http://023mrzs.com/index.php?m=cn&c=News&a=vie...,com,023mrzs.com,com.023mrzs,zho,广宗县顺发废旧物资回收有限公司\n咨询热线\n13522915401\n网站首页\n关于我们...
4,2021-08-04 16:39:13+00:00,http://030938.tclimax.com/lanmu_pc/sd_jining/i...,com,030938.tclimax.com,com.tclimax.030938,zho,中國網山東濟寧-中國網山東頻道 - 中國網·新山東 - 網上山東 | 山東新聞_四方物流\n...


In [5]:
df['tld'].value_counts().head(10).to_frame()

,tld
com,19591
org,2345
ru,2283
de,1821
net,1551
uk,1117
jp,812
it,780
pl,752
fr,712


In [6]:
df['content_languages'].value_counts().head(10).to_frame()

,content_languages
eng,17835
rus,2199
deu,1502
spa,1294
zho,1211
fra,1045
jpn,997
"jpn,eng",983
"deu,eng",905
"rus,eng",767


In [ ]:
from IPython.display import display

def print_column_size(parquet_file):
    pd.options.display.float_format = '{:,.3f}'.format
    display(column_size(parquet_file))

def column_size(parquet_file):
    compression_on = ['uncompressed', 'compressed']
    meta = pq.read_metadata(parquet_file)
    total = [0, 0]
    csize = [[0, 0] for i in range(0, len(columns))]
    for i in range(0, meta.num_row_groups):
        rg = meta.row_group(i)
        for j in range(0, rg.num_columns):
            c = rg.column(j)
            csize[j][0] += c.total_uncompressed_size
            csize[j][1] += c.total_compressed_size
            total[0] += c.total_uncompressed_size
            total[1] += c.total_compressed_size
    stats = pd.DataFrame(csize, index=columns, columns=compression_on)
    stats['ratio'] = stats['compressed'] / stats['uncompressed']
    for i in range(0, len(compression_on)):
        t = compression_on[i]
        stats['%'+t] = 100.0 * stats[t] / total[i]
    return stats


compressions = [['zstd'], ['gzip', 'gz'], ['brotli']]
sort_definitions = [[], ['tld'], ['host_reversed'], ['content_languages']]

schema = pa.schema([
    pa.field('date',                 pa.timestamp('ms')),
    pa.field('url',                  pa.string()),
    pa.field('tld',                  pa.string()),
    pa.field('host',                 pa.string()),
    pa.field('host_reversed',        pa.string()),
    pa.field('content_languages',    pa.string()),
    pa.field('content',              pa.string())
])

for compression in compressions:
    for sort_by in sort_definitions:

        compression_suffix = compression[-1]
        parquet_file = 'wet-storage-test-{}.{}.parquet'.format('-'.join(sort_by), compression_suffix)

        if sort_by:
            d = df.sort_values(sort_by)
        else:
            d = df

        # could write Parquet from Pandas
        #   d.to_parquet(parquet_data, index=False, compression=compression[0])
        # PyArrow allows for more control, eg. to pass a schema
        table = pa.Table.from_pandas(d, preserve_index=False, schema=schema)
        pq.write_table(table, parquet_file, compression=compression[0],
                       use_dictionary=['tld', 'host', 'host_reversed', 'content_languages'])

        print_file_size(parquet_file, wet_file)
        print_column_size(parquet_file)


132.439 MiB   95.9%  wet-storage-test-.zstd.parquet


,uncompressed,compressed,ratio,%uncompressed,%compressed
date,349203,143014,0.410,0.089,0.103
url,3690541,1701700,0.461,0.940,1.225
tld,51422,31721,0.617,0.013,0.023
host,929546,452112,0.486,0.237,0.326
host_reversed,929528,456309,0.491,0.237,0.329
content_languages,75519,46343,0.614,0.019,0.033
content,386427771,136030185,0.352,98.465,97.961


128.086 MiB   92.8%  wet-storage-test-tld.zstd.parquet


,uncompressed,compressed,ratio,%uncompressed,%compressed
date,349203,142629,0.408,0.089,0.106
url,3690138,1703337,0.462,0.940,1.268
tld,4235,2669,0.630,0.001,0.002
host,929784,450662,0.485,0.237,0.336
host_reversed,929766,439781,0.473,0.237,0.327
content_languages,72515,39965,0.551,0.018,0.030
content,386452059,131517303,0.340,98.477,97.931


126.027 MiB   91.3%  wet-storage-test-host_reversed.zstd.parquet


...

,uncompressed,compressed,ratio,%uncompressed,%compressed
date,349203,117473,0.336,0.089,0.114
url,3690541,1420045,0.385,0.940,1.382
tld,51422,31913,0.621,0.013,0.031
host,929546,380166,0.409,0.237,0.370
host_reversed,929528,386045,0.415,0.237,0.376
content_languages,75519,42534,0.563,0.019,0.041
content,386427771,100350233,0.260,98.465,97.685


93.597 MiB   67.8%  wet-storage-test-tld.brotli.parquet


,uncompressed,compressed,ratio,%uncompressed,%compressed
date,349203,117450,0.336,0.089,0.120
url,3690138,1414430,0.383,0.940,1.441
tld,4235,2547,0.601,0.001,0.003
host,929784,382447,0.411,0.237,0.390
host_reversed,929766,382112,0.411,0.237,0.389
content_languages,72515,35400,0.488,0.018,0.036
content,386452059,95797749,0.248,98.477,97.621


93.252 MiB   67.6%  wet-storage-test-host_reversed.brotli.parquet


,uncompressed,compressed,ratio,%uncompressed,%compressed
date,349203,117623,0.337,0.089,0.120
url,3690182,1409844,0.382,0.940,1.442
tld,4235,2547,0.601,0.001,0.003
host,929532,375068,0.404,0.237,0.384
host_reversed,929514,366002,0.394,0.237,0.374
content_languages,71281,34816,0.488,0.018,0.036
content,386450943,95464642,0.247,98.478,97.642


## Intermediate results

- (unexpectedly) brotli the compression ratio of brotli (but also gzip) is significantly than that of zstd
  - zstd
    - try to adjust the compression level?
    - use a preloaded dictionary (similar to brotli)?
- (as expected) the content column is dominant
  - retry with more homogenous content (only a single language)
  - even more homogenous sorting (eg., initially by keeping pages from one web site or even URL path prefix together)
  
Further TODOs
- benchmark also (de)compression speed
- think about "popular" filter columns, could be worth to optimize for filtering, too